In [56]:
import json
import string
import nltk
import pandas as pd
import numpy as np
import itertools

In [57]:
business = pd.read_json('business_city.json', lines=True)
review = pd.read_json('review_city.json', lines=True)
tip = pd.read_json('tip_city.json', lines=True)
user = pd.read_json('user_city.json', lines=True)

In [58]:
raw_business = pd.merge(review,business, how="left",on="business_id")
raw_business.head()

review_id                 user_id             business_id  \
0  LG2ZaYiOgpr2DK_90pYjNw  V34qejxNsCbcgD8C0HVk-Q  HQl28KMwrEKHqhFrrDqVNQ   
1  qWHp2l2lysENZObh6MAulA  5lb0POg2t-AkMFx6603cYQ  ZlCSsWS07JulSBIQlrbF5w   
2  Ne_2CSfcKIqXHmv_KC0uqg  6sJN_HlM_uwpfLJ1puf1Fg  oVuZtlCFg_zF090Nhtp0Ow   
3  2mLlQ9n5Ln_fykOuX_b0-A  EZ6g9a17UdMjtR-wrZebmA  wSAGILK2CFbq5BVZ5D67-A   
4  Ymb8ZYPg2e8dvqhdj3Gkqw  hHFSpKcnDtJOs6M-JKFZ8Q  oCvxJBOs-Mbg-4oanPO7lQ   

   stars_x  useful  funny  cool  \
0        5       1      0     0   
1        5       1      0     0   
2        4       3      0     1   
3        3       0      0     0   
4        4       0      0     0   

                                                text                date  \
0  I love Deagan's. I do. I really do. The atmosp... 2015-12-05 03:18:11   
1  Called for a 5:15 morning reservation. The dri... 2015-05-26 10:36:47   
2  A close friend was in town and so instead of t... 2018-02-01 19:15:00   
3  The service is attentive. The decor attractive... 2015-06-21 00:07:59   
4  The ingredients are always fresh and I like th... 2016-10-26 14:53:42   

                     name  ... state postal_code   latitude  longitude  \
0  Deagan's Kitchen & Bar  ...    OH       44107  41.485192 -81.800145   
1        Black Cab & Limo  ...    IL              40.116420 -88.243383   
2     Choolaah Indian BBQ  ...    PA       15206  40.460266 -79.923461   
3                   Adega  ...    OH       44115  41.499615 -81.685922   
4                Pita Pit  ...    PA       15203  40.426647 -79.965902   

   stars_y  review_count  is_open  \
0      4.0           417        1   
1      4.5            28        1   
2      4.0           302        1   
3      4.0           201        1   
4      3.5            57        1   

                                          attributes  \
0  {'BusinessAcceptsCreditCards': 'True', 'Outdoo...   
1                                {'WiFi': 'u'free''}   
2  {'RestaurantsTableService': 'False', 'Restaura...   
3  {'BusinessAcceptsCreditCards': 'True', 'Restau...   
4  {'RestaurantsPriceRange2': '1', 'RestaurantsAt...   

                                          categories  \
0  Restaurants, Gastropubs, Nightlife, American (...   
1  Airport Shuttles, Transportation, Taxis, Hotel...   
2  Restaurants, Indian, Vegetarian, Vegan, Gluten...   
3  Mediterranean, Hotels, Beer, Wine & Spirits, B...   
4  Fast Food, Greek, Breakfast & Brunch, Restaura...   

                                               hours  
0  {'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22...  
1  {'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ...  
2  {'Monday': '0:0-0:0', 'Tuesday': '11:0-17:0', ...  
3  {'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'...  
4  {'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...  

[5 rows x 22 columns]

In [59]:
Hotel_index = np.where(raw_business.categories.str.contains('Hotel').values)[0]
Hotel_business = raw_business.iloc[Hotel_index,]
Hotel_business.head()

review_id                 user_id             business_id  \
1   qWHp2l2lysENZObh6MAulA  5lb0POg2t-AkMFx6603cYQ  ZlCSsWS07JulSBIQlrbF5w   
3   2mLlQ9n5Ln_fykOuX_b0-A  EZ6g9a17UdMjtR-wrZebmA  wSAGILK2CFbq5BVZ5D67-A   
5   nQxoDWJ5kIwjYZhFILqVWQ  lJBI50awi0N8qvX4PKzH7Q  3n685UjMonF4fjBcILXkJg   
15  n2BEHit2SwqsRo6jlTagUA  abdw-Nf5xxWdi5wmUO7UYg  2qLOnQtWSlTOtkfEFyWScg   
20  mrK-GF4zYYj8A3oIlvUVTQ  wurJLbqM_iM0R64dQ8r3Qw  z9lOfnUorcbWR_xS0DSeHw   

    stars_x  useful  funny  cool  \
1         5       1      0     0   
3         3       0      0     0   
5         4       0      0     0   
15        2       2      0     0   
20        4       2      1     1   

                                                 text                date  \
1   Called for a 5:15 morning reservation. The dri... 2015-05-26 10:36:47   
3   The service is attentive. The decor attractive... 2015-06-21 00:07:59   
5   Room was very clean.  Staff was exceptionally ... 2017-08-16 11:21:42   
15  There are good things, and bad things about th... 2013-04-10 19:26:54   
20  I came for a quick bite and just ordered from ... 2014-08-24 20:25:08   

                                                name  ... state postal_code  \
1                                   Black Cab & Limo  ...    IL               
3                                              Adega  ...    OH       44115   
5   Hampton Inn Pittsburgh University/Medical Center  ...    PA       15213   
15            University of Illinois Willard Airport  ...    IL       61874   
20                                              Fugu  ...    WI       53703   

     latitude  longitude  stars_y  review_count  is_open  \
1   40.116420 -88.243383      4.5            28        1   
3   41.499615 -81.685922      4.0           201        1   
5   40.436035 -79.963010      4.0            60        1   
15  40.036489 -88.263988      4.0            38        1   
20  43.074392 -89.393664      3.5           180        1   

                                           attributes  \
1                                 {'WiFi': 'u'free''}   
3   {'BusinessAcceptsCreditCards': 'True', 'Restau...   
5   {'BusinessAcceptsCreditCards': 'True', 'Restau...   
15                                {'WiFi': 'u'free''}   
20  {'RestaurantsDelivery': 'True', 'RestaurantsAt...   

                                           categories  \
1   Airport Shuttles, Transportation, Taxis, Hotel...   
3   Mediterranean, Hotels, Beer, Wine & Spirits, B...   
5   Hotels & Travel, Event Planning & Services, Ho...   
15                          Hotels & Travel, Airports   
20  Szechuan, Thai, Chinese, Town Car Service, Tra...   

                                                hours  
1   {'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ...  
3   {'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'...  
5   {'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...  
15  {'Monday': '4:0-23:0', 'Tuesday': '4:0-23:0', ...  
20  {'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...  

[5 rows x 22 columns]

In [60]:
Hotel_Madison = Hotel_business[Hotel_business.city == 'Madison']
Hotel_Madison = Hotel_Madison.drop(columns=['city', 'state'])
Hotel_Madison.reset_index(drop=True, inplace=True)
Hotel_Madison.head()

review_id                 user_id             business_id  \
0  mrK-GF4zYYj8A3oIlvUVTQ  wurJLbqM_iM0R64dQ8r3Qw  z9lOfnUorcbWR_xS0DSeHw   
1  3s8uY7ucUBrovfujAd2TrQ  3p_YUoTXOe2aML4GbDsBdQ  3kQarjSJYYo9nGAkFjbWYg   
2  H0OlZnbjlCcLxZnUsp6p5g  g64lOV39xSLRZO0aQQ6DeQ  DHcbeJ-5tk2RQ4SdcdJqkA   
3  2YA139B8VrTjgxbAF5mLFA  vEo7fEwrEWIA4d9swwDasg  z9lOfnUorcbWR_xS0DSeHw   
4  BeaI_I2S6EJGctncfew89A  t4idL2iqySViutbKeZBXpQ  EEB9X2j6jUm2lF75LAo4Xw   

   stars_x  useful  funny  cool  \
0        4       2      1     1   
1        3       0      0     0   
2        4       1      0     1   
3        4       0      0     0   
4        1       0      0     0   

                                                text                date  \
0  I came for a quick bite and just ordered from ... 2014-08-24 20:25:08   
1  I stayed here twice in a short period of time ... 2014-06-11 17:55:35   
2  Our room on the second floor was very nice. Th... 2015-01-18 15:02:00   
3  Definitely will go back...best wonton soup I'v... 2016-07-03 19:19:40   
4  Turns out to be just as shady as uber.  They l... 2017-01-02 04:04:33   

                                              name             address  \
0                                             Fugu     411 W Gilman St   
1  Fairfield Inn & Suites by Marriott Madison East  2702 Crossroads Dr   
2                                 Hotel Ruby Marie     524 E Wilson St   
3                                             Fugu     411 W Gilman St   
4                                             Lyft                       

  postal_code   latitude  longitude  stars_y  review_count  is_open  \
0       53703  43.074392 -89.393664      3.5           180        1   
1       53718  43.133389 -89.291328      3.5            22        1   
2       53703  43.076340 -89.376209      4.0            62        1   
3       53703  43.074392 -89.393664      3.5           180        1   
4       35715  43.073052 -89.401230      2.0             4        1   

                                          attributes  \
0  {'RestaurantsDelivery': 'True', 'RestaurantsAt...   
1  {'RestaurantsPriceRange2': '2', 'WiFi': 'u'fre...   
2  {'RestaurantsPriceRange2': '2', 'BusinessAccep...   
3  {'RestaurantsDelivery': 'True', 'RestaurantsAt...   
4                                               None   

                                          categories  \
0  Szechuan, Thai, Chinese, Town Car Service, Tra...   
1  Hotels & Travel, Event Planning & Services, Ho...   
2  Bed & Breakfast, Hotels & Travel, Hotels, Even...   
3  Szechuan, Thai, Chinese, Town Car Service, Tra...   
4  Automotive, Hotels & Travel, Taxis, Transporta...   

                                               hours  
0  {'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...  
1  {'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...  
2  {'Monday': '0:0-23:59', 'Tuesday': '0:0-23:59'...  
3  {'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...  
4  {'Monday': '9:0-9:0', 'Tuesday': '9:0-9:0', 'W...

In [61]:
Hotel_Madison.shape

(4074, 20)

In [62]:
Hotel_Madison.columns

Index(['review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny',
       'cool', 'text', 'date', 'name', 'address', 'postal_code', 'latitude',
       'longitude', 'stars_y', 'review_count', 'is_open', 'attributes',
       'categories', 'hours'],
      dtype='object')

In [63]:
Hotel_Madison = Hotel_Madison.drop(columns=['business_id', 'review_id', 'user_id', 'funny', 'cool', 'date', 'latitude', 'longitude', 'is_open', 'review_count', 'hours'])
Hotel_Madison.head()

stars_x  useful                                               text  \
0        4       2  I came for a quick bite and just ordered from ...   
1        3       0  I stayed here twice in a short period of time ...   
2        4       1  Our room on the second floor was very nice. Th...   
3        4       0  Definitely will go back...best wonton soup I'v...   
4        1       0  Turns out to be just as shady as uber.  They l...   

                                              name             address  \
0                                             Fugu     411 W Gilman St   
1  Fairfield Inn & Suites by Marriott Madison East  2702 Crossroads Dr   
2                                 Hotel Ruby Marie     524 E Wilson St   
3                                             Fugu     411 W Gilman St   
4                                             Lyft                       

  postal_code  stars_y                                         attributes  \
0       53703      3.5  {'RestaurantsDelivery': 'True', 'RestaurantsAt...   
1       53718      3.5  {'RestaurantsPriceRange2': '2', 'WiFi': 'u'fre...   
2       53703      4.0  {'RestaurantsPriceRange2': '2', 'BusinessAccep...   
3       53703      3.5  {'RestaurantsDelivery': 'True', 'RestaurantsAt...   
4       35715      2.0                                               None   

                                          categories  
0  Szechuan, Thai, Chinese, Town Car Service, Tra...  
1  Hotels & Travel, Event Planning & Services, Ho...  
2  Bed & Breakfast, Hotels & Travel, Hotels, Even...  
3  Szechuan, Thai, Chinese, Town Car Service, Tra...  
4  Automotive, Hotels & Travel, Taxis, Transporta...

In [64]:
Hotel_Madison.shape

(4074, 9)

In [65]:
text = list(Hotel_Madison.text)

In [66]:
text = [a.lower() for a in text]
text = ["".join([char for char in a if char not in string.punctuation]) for a in text]

In [67]:
words = [nltk.word_tokenize(a) for a in text]

In [68]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
filtered_words = [[word for word in a if word not in stop_words] for a in words]

In [69]:
from nltk.stem import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [70]:
lemmatized_words = [[lmtzr.lemmatize(x) for x in a] for a in filtered_words]

In [71]:
word_list = list(itertools.chain.from_iterable(lemmatized_words))
from collections import Counter
freq = pd.DataFrame.from_dict(Counter(word_list), orient='index')

In [72]:
freq.sort_values(by=0, ascending=False)

0
room        4583
hotel       3557
stay        1802
time        1773
great       1740
...          ...
throughput     1
swamped        1
fixwhen        1
excusedid      1
deviate        1

[15564 rows x 1 columns]

In [73]:
dictionary = list(freq[freq[0] > 10].index)

In [74]:
word_embedding = np.zeros([len(lemmatized_words), len(dictionary)])

In [75]:
for i,x in enumerate(lemmatized_words):
    for j,y in enumerate(dictionary):
        word_embedding[i,j] = sum([z == y for z in x])

In [76]:
word_embedding

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [77]:
df = pd.concat([Hotel_Madison, pd.DataFrame(word_embedding, columns=dictionary)], axis=1)

In [78]:
df

stars_x  useful                                               text  \
0           4       2  I came for a quick bite and just ordered from ...   
1           3       0  I stayed here twice in a short period of time ...   
2           4       1  Our room on the second floor was very nice. Th...   
3           4       0  Definitely will go back...best wonton soup I'v...   
4           1       0  Turns out to be just as shady as uber.  They l...   
...       ...     ...                                                ...   
4069        1       1  I have never posted a negative review on any s...   
4070        1       0  I rent 200 plus days a year. Was offered an Im...   
4071        5       1  This was one of the coolest hotels I've stayed...   
4072        1       0  The worst National I've been to across the who...   
4073        5       1  Such a great experience from check in to check...   

                                                 name                address  \
0                                                Fugu        411 W Gilman St   
1     Fairfield Inn & Suites by Marriott Madison East     2702 Crossroads Dr   
2                                    Hotel Ruby Marie        524 E Wilson St   
3                                                Fugu        411 W Gilman St   
4                                                Lyft                          
...                                               ...                    ...   
4069                  Americas Best Value Inn Madison     3438 Highway 12-18   
4070                              National Car Rental  4000 International Ln   
4071                    Hotel Indigo Madison Downtown   901 E Washington Ave   
4072                              National Car Rental  4000 International Ln   
4073                    Hotel Indigo Madison Downtown   901 E Washington Ave   

     postal_code  stars_y                                         attributes  \
0          53703      3.5  {'RestaurantsDelivery': 'True', 'RestaurantsAt...   
1          53718      3.5  {'RestaurantsPriceRange2': '2', 'WiFi': 'u'fre...   
2          53703      4.0  {'RestaurantsPriceRange2': '2', 'BusinessAccep...   
3          53703      3.5  {'RestaurantsDelivery': 'True', 'RestaurantsAt...   
4          35715      2.0                                               None   
...          ...      ...                                                ...   
4069       53704      1.5  {'RestaurantsPriceRange2': '1', 'WiFi': 'u'fre...   
4070       53704      2.0                                               None   
4071       53703      5.0  {'BusinessAcceptsBitcoin': 'False', 'Wheelchai...   
4072       53704      2.0                                               None   
4073       53703      5.0  {'BusinessAcceptsBitcoin': 'False', 'Wheelchai...   

                                             categories  came  ...  getz  \
0     Szechuan, Thai, Chinese, Town Car Service, Tra...   1.0  ...   0.0   
1     Hotels & Travel, Event Planning & Services, Ho...   0.0  ...   0.0   
2     Bed & Breakfast, Hotels & Travel, Hotels, Even...   0.0  ...   0.0   
3     Szechuan, Thai, Chinese, Town Car Service, Tra...   0.0  ...   0.0   
4     Automotive, Hotels & Travel, Taxis, Transporta...   0.0  ...   0.0   
...                                                 ...   ...  ...   ...   
4069  Hotels & Travel, Hotels, Event Planning & Serv...   0.0  ...   0.0   
4070                        Car Rental, Hotels & Travel   0.0  ...   0.0   
4071  Venues & Event Spaces, Hotels & Travel, Hotels...   0.0  ...   0.0   
4072                        Car Rental, Hotels & Travel   0.0  ...   0.0   
4073  Venues & Event Spaces, Hotels & Travel, Hotels...   0.0  ...   0.0   

      llama  sheep  sampled  tsa  precheck  flown  ord  americinn  arbor  
0       0.0    0.0      0.0  0.0       0.0    0.0  0.0        0.0    0.0  
1       0.0    0.0      0.0  0.0       0.0    0.0  0.0        0.0    0.0  
2       0.0    0.0      0.0  0.0 

In [79]:
df.to_csv('df.csv')

In [80]:
raw_tip = pd.merge(user,tip, how="left",on="user_id")
raw_tip.head()

user_id          name  review_count        yelping_since  \
0  xvu8G900tezTzbbfqmTKvA          Anne           485  2008-08-09 00:30:27   
1  z5_82komKV3mI4ASGe2-FQ         Steve           186  2007-02-27 07:09:40   
2  QG13XBbgHWydzThRBGJtyw        Claire           218  2007-06-04 00:14:10   
3  I_6wY8_RsewziNnKhGZg4g          Jeff           405  2010-08-05 18:42:29   
4  1WBxJ2r3A2QYfRSEzgcmkQ  Evan Thoreau           315  2009-10-30 15:05:44   

   useful  funny  cool                                         elite  \
0    1265    400   512  2009,2010,2011,2012,2014,2015,2016,2017,2018   
1     642    192   155                                          2007   
2     587    372   426                                     2008,2009   
3     799    244   312       2010,2011,2012,2013,2014,2015,2016,2017   
4     658    251   361            2010,2011,2012,2013,2014,2015,2016   

                                             friends  fans  ...  \
0  3W3ZMSthojCUirKEqAwGNw, eTIbuu23j9tOgmIa9POyLQ...    78  ...   
1  E-fXXmALnZTPmzzL6rV4cQ, L2g6vS1GDfqLEpiDOFfCkA...    15  ...   
2  tnfVwTpuPlR6W2xEAMiqsg, Bd_16T8Dva27lF5LIrUmcQ...    31  ...   
3  OGa2q6LvuQLYFUG_RCcALw, 0uM2R-uMBemiNlrFG5j56A...    13  ...   
4  JUsNVy1vopeN61geWX6qRw, KDrRwgcGfMHzJXACVtreMA...    31  ...   

   compliment_note  compliment_plain  compliment_cool  compliment_funny  \
0               22                28               31                31   
1               60                36               32                32   
2               18                24               37                37   
3               19                48               76                76   
4               52               151               50                50   

   compliment_writer  compliment_photos             business_id  \
0                 19                 31  q6dOVAsEZ0_JeVcuq1htIw   
1                  8                  9                     NaN   
2                 17                  5                     NaN   
3                 14                  6  _iuNOWWFHtCuQFpBYTjXkg   
4                 33                  5                     NaN   

                                      text                date  \
0                        24 hour Starbucks 2014-06-28 03:02:24   
1                                      NaN                 NaT   
2                                      NaN                 NaT   
3  Easy out, easy in. Price is a bit high. 2011-01-13 14:08:13   
4                                      NaN                 NaT   

   compliment_count  
0               0.0  
1               NaN  
2               NaN  
3               0.0  
4               NaN  

[5 rows x 26 columns]

In [81]:
useful_tip = raw_tip[raw_tip.useful > 100]
tip_business = pd.merge(business,useful_tip, how="left",on="business_id")
tip_business.head()

business_id                        name_x  \
0  pQeaRpvuhoEqudo3uymHIQ           The Empanadas House   
1  7Dv4_HAxsxvadEsT5fxQBg   Dependable Brakes & Exhaust   
2  M_guz7Dj7hX0evS672wIwA    Chocolate Shoppe Ice Cream   
3  03x6ZlJ7s39DHqfTJU1Ecg        Beechmont Country Club   
4  _4Oe9V-qTpU5iemM9bphlA  Goodyear Auto Service Center   

                         address        city state postal_code   latitude  \
0                 404 E Green St   Champaign    IL       61820  40.110446   
1         1110 Saw Mill Run Blvd  Pittsburgh    PA       15226  40.406667   
2              2831 Parmenter St   Middleton    WI       53562  43.105310   
3             29600 Chagrin Blvd   Beachwood    OH       44122  41.461448   
4  800 Fort Duquesne Blvd, Ste 1  Pittsburgh    PA       15222  40.444614   

   longitude  stars  review_count_x  ...  compliment_list compliment_note  \
0 -88.233073    4.5               5  ...              NaN             NaN   
1 -80.004450    5.0               5  ...              NaN             NaN   
2 -89.510142    3.5               6  ...              NaN             NaN   
3 -81.470127    4.5               7  ...              0.0             7.0   
4 -79.999344    3.5              12  ...              0.0             3.0   

  compliment_plain compliment_cool compliment_funny compliment_writer  \
0              NaN             NaN              NaN               NaN   
1              NaN             NaN              NaN               NaN   
2              NaN             NaN              NaN               NaN   
3             11.0             6.0              6.0               8.0   
4             21.0            14.0             14.0               4.0   

   compliment_photos                                               text  \
0                NaN                                                NaN   
1                NaN                                                NaN   
2                NaN                                                NaN   
3                0.0                                    Brad's wedding!   
4                0.0  Ask for quote and check with a mother dealersh...   

                 date  compliment_count  
0                 NaT               NaN  
1                 NaT               NaN  
2                 NaT               NaN  
3 2012-05-27 22:43:22               0.0  
4 2016-03-29 20:37:31               0.0  

[5 rows x 39 columns]

In [82]:
tip_index = np.where(tip_business.categories.str.contains('Hotel').values)[0]
Hotel_tip = tip_business.iloc[tip_index,]
Hotel_tip.head()

business_id                                        name_x  \
107  0JOuIE9U2NJ34Plkrx2k-Q                Wayne Ace Bus And Limo Service   
123  B_wz5DFPuYBLedrk8XW6jA                           Holiday Inn Express   
159  RqwuaHAybTqbkmGe1RZLXQ  Embassy Suites by Hilton Cleveland Beachwood   
160  RqwuaHAybTqbkmGe1RZLXQ  Embassy Suites by Hilton Cleveland Beachwood   
161  RqwuaHAybTqbkmGe1RZLXQ  Embassy Suites by Hilton Cleveland Beachwood   

               address       city state postal_code   latitude  longitude  \
107   1219 County Rd D     Oregon    WI       53575  42.926112 -89.384563   
123     2850 Medina Rd     Medina    OH       44256  41.135395 -81.790136   
159  3775 Park East Dr  Beachwood    OH       44122  41.457708 -81.492653   
160  3775 Park East Dr  Beachwood    OH       44122  41.457708 -81.492653   
161  3775 Park East Dr  Beachwood    OH       44122  41.457708 -81.492653   

     stars  review_count_x  ...  compliment_list compliment_note  \
107    3.0              11  ...              NaN             NaN   
123    4.0               3  ...              NaN             NaN   
159    3.0              71  ...              0.0            75.0   
160    3.0              71  ...             10.0           144.0   
161    3.0              71  ...              0.0             4.0   

    compliment_plain compliment_cool compliment_funny compliment_writer  \
107              NaN             NaN              NaN               NaN   
123              NaN             NaN              NaN               NaN   
159             64.0           100.0            100.0             134.0   
160            305.0           330.0            330.0             152.0   
161              8.0            14.0             14.0               9.0   

     compliment_photos                            text                date  \
107                NaN                             NaN                 NaT   
123                NaN                             NaN                 NaT   
159               13.0  Free breakfast and happy hour! 2012-12-05 21:36:03   
160               46.0          Cleveland's Got Sole 3 2012-06-24 18:44:33   
161                0.0                        Wedding! 2012-03-17 23:04:45   

     compliment_count  
107               NaN  
123               NaN  
159               0.0  
160               0.0  
161               0.0  

[5 rows x 39 columns]

In [83]:
Hotel_tip.columns

Index(['business_id', 'name_x', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count_x', 'is_open',
       'attributes', 'categories', 'hours', 'user_id', 'name_y',
       'review_count_y', 'yelping_since', 'useful', 'funny', 'cool', 'elite',
       'friends', 'fans', 'average_stars', 'compliment_hot', 'compliment_more',
       'compliment_profile', 'compliment_cute', 'compliment_list',
       'compliment_note', 'compliment_plain', 'compliment_cool',
       'compliment_funny', 'compliment_writer', 'compliment_photos', 'text',
       'date', 'compliment_count'],
      dtype='object')

In [84]:
Hotel_tip = Hotel_tip.drop(columns=['business_id', 'name_x', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'review_count_x', 'is_open',
       'attributes', 'categories', 'hours', 'user_id', 'name_y',
       'review_count_y', 'yelping_since', 'useful', 'funny', 'cool', 'elite',
       'friends', 'fans', 'average_stars', 'compliment_hot', 'compliment_more',
       'compliment_profile', 'compliment_cute', 'compliment_list',
       'compliment_note', 'compliment_plain', 'compliment_cool',
       'compliment_funny', 'compliment_writer', 'compliment_photos', 'date', 'compliment_count'])
Hotel_tip.head()

stars                            text
107    3.0                             NaN
123    4.0                             NaN
159    3.0  Free breakfast and happy hour!
160    3.0          Cleveland's Got Sole 3
161    3.0                        Wedding!

In [85]:
Hotel_tip = Hotel_tip.dropna()
Hotel_tip.head()

stars                                               text
159    3.0                     Free breakfast and happy hour!
160    3.0                             Cleveland's Got Sole 3
161    3.0                                           Wedding!
162    3.0  Be sure to stop by and have an omelette prepar...
225    3.0  Use promo "sffby" for a $30 credit if you're a...

In [88]:
text_tip = list(Hotel_tip.text)
text_tip = [a.lower() for a in text_tip]
text_tip = ["".join([char for char in a if char not in string.punctuation]) for a in text_tip]
words_tip = [nltk.word_tokenize(a) for a in text_tip]
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
filtered_words_tip = [[word for word in a if word not in stop_words] for a in words_tip]
from nltk.stem import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
lemmatized_words_tip = [[lmtzr.lemmatize(x) for x in a] for a in filtered_words_tip]
word_list_tip = list(itertools.chain.from_iterable(lemmatized_words_tip))
from collections import Counter
freq_tip = pd.DataFrame.from_dict(Counter(word_list_tip), orient='index')
freq_tip.sort_values(by=0, ascending=False)
dictionary_tip = list(freq_tip[freq_tip[0] > 10].index)
word_embedding_tip = np.zeros([len(lemmatized_words_tip), len(dictionary_tip)])
for i,x in enumerate(lemmatized_words_tip):
    for j,y in enumerate(dictionary_tip):
        word_embedding_tip[i,j] = sum([z == y for z in x])
word_embedding_tip

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [102]:
Tip_star=Hotel_tip.stars
Tip_star.head()

0    3.0
1    3.0
2    3.0
3    3.0
4    3.0
Name: stars, dtype: float64

In [104]:
df_tip = pd.concat([Tip_star, pd.DataFrame(word_embedding_tip, columns=dictionary_tip)], axis=1)
df_tip.head()

stars  free  breakfast  happy  hour  cleveland  got    3  sure  stop  ...  \
0    3.0   1.0        1.0    1.0   1.0        0.0  0.0  0.0   0.0   0.0  ...   
1    3.0   0.0        0.0    0.0   0.0        1.0  1.0  1.0   0.0   0.0  ...   
2    3.0   0.0        0.0    0.0   0.0        0.0  0.0  0.0   0.0   0.0  ...   
3    3.0   0.0        1.0    0.0   0.0        0.0  1.0  0.0   1.0   1.0  ...   
4    3.0   0.0        0.0    0.0   0.0        0.0  0.0  0.0   0.0   0.0  ...   

   property  renovation  comfy  garage  warm  excellent  another  special  \
0       0.0         0.0    0.0     0.0   0.0        0.0      0.0      0.0   
1       0.0         0.0    0.0     0.0   0.0        0.0      0.0      0.0   
2       0.0         0.0    0.0     0.0   0.0        0.0      0.0      0.0   
3       0.0         0.0    0.0     0.0   0.0        0.0      0.0      0.0   
4       0.0         0.0    0.0     0.0   0.0        0.0      0.0      0.0   

   change  alternate  
0     0.0        0.0  
1     0.0        0.0  
2     0.0        0.0  
3     0.0        0.0  
4     0.0        0.0  

[5 rows x 256 columns]

In [ ]:
df_tip.to_csv('df_tip.csv')